<a href="https://colab.research.google.com/github/Eezzeldin/candy/blob/main/Candy_IntegerProgramming_searchspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
With Continous Variables

                   Adjusted Balanced Accuracy
chocolate                           0.972973
fruity                              0.973684
caramel                             0.129779
peanutyalmondy                      0.143863
nougat                              0.558608
crispedricewafer                    0.272894
hard                                0.085714
bar                                 0.889881
pluribus                            0.512195
other                               1.000000


                  Adjusted Balanced Accuracy
chocolate                           0.972973
fruity                              0.973684
caramel                             0.188129
peanutyalmondy                      0.373239
nougat                              0.688645
crispedricewafer                    0.390110
hard                                0.190476
bar                                 0.889881
pluribus                            0.489468
other                               1.000000



peanutyalmondy       0.143863 - 0.373239 = 0.229376
nougat               0.558608 - 0.688645 = 0.13
crispedricewafer     0.272894 - 0.390110 = 0.1172
hard                 0.085714 - 0.190476 = 0.104
caramel              0.129779 - 0.188129 = 0.05835



peanutyalmondy
nougat
crispedricewafer
hard
caramel

pluribus
bar
'''

In [26]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = '/content/ate_df_xyz123.csv'
df = pd.read_csv(file_path)
df

,Unnamed: 0,winpercent,pricepercent,sugarpercent,treatments
0,0,0.187205,0.290580,0.058453,chocolate
1,1,0.087560,0.198220,0.200239,caramel
2,2,0.171796,0.232638,0.082415,peanutyalmondy
3,3,0.110732,0.151184,0.127181,nougat
4,4,0.184654,0.352849,0.031096,crispedricewafer
5,5,-0.124270,-0.174848,0.083500,hard
6,6,-0.067629,-0.128992,0.024898,pluribus
7,7,-0.112572,-0.244271,-0.012242,fruity
8,8,-0.161970,-0.124387,-0.088911,other_candy


In [27]:

# Remove the specified columns: fruity, other, pluribus
columns_to_drop = ['fruity', 'other_candy', 'pluribus']
columns_to_drop = [""]
c          = df ["treatments"].isin (columns_to_drop)
df         = df [~c]
df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')

# Display the first few rows of the cleaned DataFrame
df_cleaned


,Unnamed: 0,winpercent,pricepercent,sugarpercent,treatments
0,0,0.187205,0.290580,0.058453,chocolate
1,1,0.087560,0.198220,0.200239,caramel
2,2,0.171796,0.232638,0.082415,peanutyalmondy
3,3,0.110732,0.151184,0.127181,nougat
4,4,0.184654,0.352849,0.031096,crispedricewafer
5,5,-0.124270,-0.174848,0.083500,hard
6,6,-0.067629,-0.128992,0.024898,pluribus
7,7,-0.112572,-0.244271,-0.012242,fruity
8,8,-0.161970,-0.124387,-0.088911,other_candy


In [28]:
from sklearn.preprocessing import LabelEncoder

# Encode the 'treatments' column to numeric values
label_encoder = LabelEncoder()
df_cleaned['treatments_encoded'] = label_encoder.fit_transform(df_cleaned['treatments'])

# Extract the encoded values
encoded_values = df_cleaned['treatments_encoded'].unique()

# Create a mapping from treatments to encoded integers
treatment_mapping = {label: idx for idx, label in enumerate(label_encoder.classes_)}

# Generate the search space: one-hot encoded representation
search_space = pd.get_dummies(df_cleaned['treatments_encoded'])

# Add the treatment labels for clarity
search_space.columns = label_encoder.inverse_transform(search_space.columns)

# Display the search space
search_space


,caramel,chocolate,crispedricewafer,fruity,hard,nougat,other_candy,peanutyalmondy,pluribus
0,False,True,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,True,False,False,False
4,False,False,True,False,False,False,False,False,False
5,False,False,False,False,True,False,False,False,False
6,False,False,False,False,False,False,False,False,True
7,False,False,False,True,False,False,False,False,False
8,False,False,False,False,False,False,True,False,False


In [29]:
search_space.shape

(9, 9)

In [30]:
import itertools

# Get the list of unique treatments
unique_treatments = label_encoder.classes_
n_treatments = len(unique_treatments)

# Generate all possible binary combinations for treatments
binary_combinations = list(itertools.product([0, 1], repeat=n_treatments))

# Create a DataFrame from the binary combinations
binary_search_space = pd.DataFrame(binary_combinations, columns=unique_treatments)

# Display the first few rows of the binary search space
binary_search_space


,caramel,chocolate,crispedricewafer,fruity,hard,nougat,other_candy,peanutyalmondy,pluribus
0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,1,1
4,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
507,1,1,1,1,1,1,0,1,1
508,1,1,1,1,1,1,1,0,0
509,1,1,1,1,1,1,1,0,1
510,1,1,1,1,1,1,1,1,0


In [31]:
binary_search_space.shape

(512, 9)

In [32]:
# Adjust filter function to consider only treatment columns
def filter_rows_encoded(df, combination, label_encoder):
    mask = True
    for treatment, value in combination.items():
        if treatment in label_encoder.classes_:
            encoded_treatment = label_encoder.transform([treatment])[0]
            mask &= (df['treatments_encoded'] == encoded_treatment) == value
    return df[mask]

# Adjust computation to compute aggregate values
for index, combination in binary_search_space.iterrows():
    subset_df = filter_rows_encoded(df_cleaned, combination, label_encoder)

    binary_search_space.loc[index, 'winpercent'] = subset_df['winpercent'].mean() if not subset_df.empty else 0
    binary_search_space.loc[index, 'pricepercent'] = subset_df['pricepercent'].mean() if not subset_df.empty else 0
    binary_search_space.loc[index, 'sugarpercent'] = subset_df['sugarpercent'].mean() if not subset_df.empty else 0

# Display the updated search space with aggregate values
binary_search_space.head()


,caramel,chocolate,crispedricewafer,fruity,hard,nougat,other_candy,peanutyalmondy,pluribus,winpercent,pricepercent,sugarpercent
0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
1,0,0,0,0,0,0,0,0,1,-0.067629,-0.128992,0.024898
2,0,0,0,0,0,0,0,1,0,0.171796,0.232638,0.082415
3,0,0,0,0,0,0,0,1,1,0.000000,0.000000,0.000000
4,0,0,0,0,0,0,1,0,0,-0.161970,-0.124387,-0.088911


In [39]:
# Adjusted computation to multiply binary decisions by respective parameters

# Create dictionaries to hold aggregate values for each treatment
winpercent_dict = df_cleaned.groupby('treatments_encoded')['winpercent'].mean().to_dict()
pricepercent_dict = df_cleaned.groupby('treatments_encoded')['pricepercent'].mean().to_dict()
sugarpercent_dict = df_cleaned.groupby('treatments_encoded')['sugarpercent'].mean().to_dict()

# Recalculate aggregate values
for index, combination in binary_search_space.iterrows():
    winpercent_sum = 0
    pricepercent_sum = 0
    sugarpercent_sum = 0

    for treatment, value in combination.items():
        if treatment in label_encoder.classes_ and value == 1:
            encoded_treatment = label_encoder.transform([treatment])[0]
            winpercent_sum += winpercent_dict.get(encoded_treatment, 0)
            pricepercent_sum += pricepercent_dict.get(encoded_treatment, 0)
            sugarpercent_sum += sugarpercent_dict.get(encoded_treatment, 0)

    binary_search_space.loc[index, 'winpercent'] = winpercent_sum
    binary_search_space.loc[index, 'pricepercent'] = pricepercent_sum
    binary_search_space.loc[index, 'sugarpercent'] = sugarpercent_sum

# Display the updated search space with corrected aggregate values
binary_search_space.head()


,caramel,chocolate,crispedricewafer,fruity,hard,nougat,other_candy,peanutyalmondy,pluribus,winpercent,pricepercent,sugarpercent
0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
1,0,0,0,0,0,0,0,0,1,-0.067629,-0.128992,0.024898
2,0,0,0,0,0,0,0,1,0,0.171796,0.232638,0.082415
3,0,0,0,0,0,0,0,1,1,0.104167,0.103645,0.107313
4,0,0,0,0,0,0,1,0,0,-0.161970,-0.124387,-0.088911


In [40]:
effective_search_space_c1 = binary_search_space ["winpercent"]   == 0
effective_search_space_c2 = binary_search_space ["sugarpercent"] == 0
effective_search_space_c3 = binary_search_space ["pricepercent"] == 0
effective_search_space_c  = effective_search_space_c1 & effective_search_space_c2 & effective_search_space_c3
effective_search_space    = binary_search_space [~effective_search_space_c ]
effective_search_space

,caramel,chocolate,crispedricewafer,fruity,hard,nougat,other_candy,peanutyalmondy,pluribus,winpercent,pricepercent,sugarpercent
1,0,0,0,0,0,0,0,0,1,-0.067629,-0.128992,0.024898
2,0,0,0,0,0,0,0,1,0,0.171796,0.232638,0.082415
3,0,0,0,0,0,0,0,1,1,0.104167,0.103645,0.107313
4,0,0,0,0,0,0,1,0,0,-0.161970,-0.124387,-0.088911
5,0,0,0,0,0,0,1,0,1,-0.229599,-0.253379,-0.064014
...,...,...,...,...,...,...,...,...,...,...,...,...
507,1,1,1,1,1,1,0,1,1,0.437476,0.677359,0.595540
508,1,1,1,1,1,1,1,0,0,0.171339,0.449327,0.399316
509,1,1,1,1,1,1,1,0,1,0.103710,0.320334,0.424213
510,1,1,1,1,1,1,1,1,0,0.343135,0.681965,0.481731


In [41]:
effective_search_space.shape

(511, 12)

In [43]:
effective_search_space.to_csv ("effective_search_space_xyz123.csv",index= False)